In [2]:
!pip install plotly
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 21.0 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
#from utils import get_data_dim, get_series_color, get_y_height
import numpy as np
import os
import json
from datetime import datetime
import plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from dataset import SlidingWindowDataset
import plotly.express as px
import plotly.graph_objects as go
import wandb
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler

In [ ]:

wandb.init(project="AnomalyDetection")
table = wandb.Table(columns = ["mtad_gat"])


In [3]:
df = pd.read_pickle("./results/train_output.pkl")
df2 = pd.read_pickle("./results/test_output.pkl")

df.to_csv("./results/res_train.csv", sep=";", index=False)
df2.to_csv("./results/res_test.csv", sep=";", index=False)

In [4]:
df = pd.read_csv("./data/merged_annuale_meteo_sunrise_filtrato15_comm_code.csv", delimiter=";")
df_res = pd.read_csv("./results/res_train.csv", delimiter=";")
df_res_test = pd.read_csv("./results/res_test.csv", delimiter=";")

timestamp = df["timestamp"].copy(deep=True)
timestamp_t = df["timestamp"].copy(deep=True)
df.drop(columns={"timestamp"}, inplace=True)
timestamp_t.drop(index=timestamp_t.index[:28604], axis=0, inplace=True)
timestamp.drop(index=timestamp.index[:100], axis=0, inplace=True)
timestamp_t.reset_index(drop=True)
timestamp.reset_index(drop=True)



df_train = df.drop(index=df.index[:100], axis=0)
df_train.drop(index=df_train.index[28604:], axis=0, inplace=True)
df_test = df.drop(index=df.index[:28804], axis=0)

In [54]:
print(len(df_res.index))
print(len(df_train.index))

28604
28604


In [57]:
print(len(df_res_test.index))
print(len(df_test.index))

7077
7077


In [97]:
for i, col in enumerate(df.columns):
    print(f" index = {i}, name = {col}")

 index = 0, name = INV03_CurrentDC(A)
 index = 1, name = INV03_CurrentAC(A)
 index = 2, name = INV03_PowerAC(kW)
 index = 3, name = INV03_PowerDC(kW)
 index = 4, name = INV03_InternalTemperature(C)
 index = 5, name = INV03_HeatSinkTemperature(C)
 index = 6, name = INV03_VoltageDC(V)
 index = 7, name = INV03_VoltageAC(V)
 index = 8, name = INV02_CurrentDC(A)
 index = 9, name = INV02_CurrentAC(A)
 index = 10, name = INV02_PowerAC(kW)
 index = 11, name = INV02_PowerDC(kW)
 index = 12, name = INV02_InternalTemperature(C)
 index = 13, name = INV02_HeatSinkTemperature(C)
 index = 14, name = INV02_VoltageDC(V)
 index = 15, name = INV02_VoltageAC(V)
 index = 16, name = SOL_Irradiance(W/m2)
 index = 17, name = SOL_ModuleTemperature(C)
 index = 18, name = Cont_TotalEnergy(kWh)
 index = 19, name = Impianto_SolargisGHI(W/m2)
 index = 20, name = Impianto_SolargisGTI(W/m2)
 index = 21, name = minute_sin
 index = 22, name = minute_cos
 index = 23, name = hour_sin
 index = 24, name = hour_cos
 index =

In [5]:
for i, col in enumerate(df_res.columns):
    print(f" index = {i}, name = {col}")

 index = 0, name = Forecast_0
 index = 1, name = Recon_0
 index = 2, name = True_0
 index = 3, name = A_Score_0
 index = 4, name = Forecast_1
 index = 5, name = Recon_1
 index = 6, name = True_1
 index = 7, name = A_Score_1
 index = 8, name = Forecast_2
 index = 9, name = Recon_2
 index = 10, name = True_2
 index = 11, name = A_Score_2
 index = 12, name = Forecast_3
 index = 13, name = Recon_3
 index = 14, name = True_3
 index = 15, name = A_Score_3
 index = 16, name = Forecast_4
 index = 17, name = Recon_4
 index = 18, name = True_4
 index = 19, name = A_Score_4
 index = 20, name = Forecast_5
 index = 21, name = Recon_5
 index = 22, name = True_5
 index = 23, name = A_Score_5
 index = 24, name = Forecast_6
 index = 25, name = Recon_6
 index = 26, name = True_6
 index = 27, name = A_Score_6
 index = 28, name = Forecast_7
 index = 29, name = Recon_7
 index = 30, name = True_7
 index = 31, name = A_Score_7
 index = 32, name = Forecast_8
 index = 33, name = Recon_8
 index = 34, name = Tru

# Train plot

In [4]:
def get_corr(df:pd.DataFrame, df_n:pd.DataFrame):
    
    mapper = {}
    for col in df:
        if "Score" in col:
            an_name = col.split("_")
            if an_name[-1]=="Global":
                pass
            else:
                mapper[df_n.columns[int(an_name[-1])]] = df[col].values
    
    an_df = pd.DataFrame.from_dict(mapper)
 
    sorted_arr = []
    for index, value in df["A_Pred_Global"].items(): 
        unsorted_arr=[]
        if value==1:
            value_arr = []
            
            for i, col in an_df.items():
                value_arr.append([i,an_df.loc[index, i]])
            unsorted_arr=value_arr
            #print(unsorted_arr[0][1][1])
            unsorted_arr=sorted(unsorted_arr, key=lambda x: x[1], reverse=True)
            sorted_arr.append([unsorted_arr, index])
            #print(an_df.iloc[index].to_numpy())
            #np.append(sorted_arr,an_df.iloc[index].to_numpy())
            #sorted_arr.append(an_df.iloc[index].to_numpy())
            #sorted_arr = np.array(value_arr)
            
    return np.array(sorted_arr, dtype=object)
                    

In [5]:
#dato un punto di anomalia, cerca tutti quelli vicini
def anomaly_offset(df:pd.DataFrame, index, col):
    offset = 1
    while df_res[col].iloc[index] == 1:
        offset += 1
        index+=1
    return offset

In [9]:
def print_plot(df_res:pd.DataFrame, df:pd.DataFrame, timestamp:pd.DataFrame, path:str):
    sorted_arr = get_corr(df_res, df)
    
    for i, col in enumerate(df.columns):
        #wandb.init(project="AnomalyDetection")
        #table = wandb.Table(columns = ["mtad_gat"])
        if i<=20:
            #fig = make_subplots(specs=[[{"secondary_y": True}]])

            fig = go.Figure()

            #print rectangle if local anomaly = 1
            index=0
            while index< len(df_res.index):
                if df_res[f"A_Pred_{i}"].iloc[index] == 1:
                    offset = 1
                    offset = anomaly_offset(df_res, index, f"A_Pred_{i}")
                    fig.add_shape(
                        type="rect",
                        x0=timestamp.iloc[index-1],
                        y0=0,
                        x1=timestamp.iloc[index+offset],
                        y1=1.2,
                        line=dict(
                        color="rgba(0,0,0,0)",
                        width=1,
                        ),
                        fillcolor="rgba(0,0,255,0.3)"
                        )
                    index += offset
                    continue
                index+=1


            #print rectangle if globalanomaly = 1
            index=0
            while index< len(df_res.index):
                if df_res["A_Pred_Global"].iloc[index] == 1:
                    offset = 1
                    offset = anomaly_offset(df_res, index, "A_Pred_Global")
                    fig.add_shape(
                        type="rect",
                        x0=timestamp.iloc[index],
                        y0=0,
                        x1=timestamp.iloc[index+offset],
                        y1=1.2,
                        line=dict(
                        color="rgba(0,0,0,0)",
                        width=1,
                        ),
                        fillcolor="rgba(255, 0, 0, 0.3)"
                        )
                    index += offset
                    continue

                index+=1

            a_i = 0
            for ind in df_res["A_Pred_Global"].index:
                if a_i>(len(sorted_arr)-1):
                    break
                if ind == sorted_arr[a_i][1]:
                    corr = f"{sorted_arr[a_i][0][0]}<br>{sorted_arr[a_i][0][1]} <br> {sorted_arr[a_i][0][2]} <br> {sorted_arr[a_i][0][3]} <br> {sorted_arr[a_i][0][4]}"

                    fig.add_scatter(x=[timestamp.values[ind]], y=[df_res[f"True_{i}"].values[ind]], hovertext=corr, marker=dict(size=8, color="rgba(255,0,0,0)"), showlegend=False)
                    a_i+=1

            fig.add_trace(go.Scatter(x=timestamp.values, y=df_res[f"Forecast_{i}"], name=f"{col} forecast",  line=dict(dash='dash', color="midnightblue")))
            fig.add_trace(go.Scatter(x=timestamp.values, y=df_res[f"Recon_{i}"], name=f"{col} recon",  line=dict(dash='dash', color="forestgreen")))
            fig.add_trace(go.Scatter(x=timestamp.values, y=df_res[f"True_{i}"], name=f"{col} true", line=dict(color="firebrick")))

            fig.add_trace(go.Scatter(x=timestamp.values, y=df[f"INV03_CommunicationCode_60"], name=f"INV03 comm code"))
            fig.add_trace(go.Scatter(x=timestamp.values, y=df[f"INV02_CommunicationCode_60"], name=f"INV02 comm code"))
            fig.add_trace(go.Scatter(x=timestamp.values, y=df[f"Cont_CommunicationCode_60"], name=f"Cont comm code"))


            fig.update_layout(go.Layout(title= f"{col} Plot", xaxis=dict(title='timestamp'), yaxis=dict(title='value')))

            fig.write_html(f"{path}/{i}_plot_mtad_gat.html", auto_play = False)
        else:
            break

In [10]:
print_plot(df_res, df_train, timestamp, "./graphs/15_code_train")

# Test plot

In [11]:
print_plot(df_res_test, df_test, timestamp_t, "./graphs/15_code_test")

In [8]:
code_df = pd.read_csv("./data/comm_code.csv", delimiter=";")

In [9]:
print(code_df)

        INV01_CommunicationCode  INV03_CommunicationCode  \
0                            60                       60   
1                            60                       60   
2                            60                       60   
3                            60                       60   
4                            60                       60   
...                         ...                      ...   
143425                       60                        0   
143426                       60                        0   
143427                       60                        0   
143428                       60                        0   
143429                       60                        0   

        INV02_CommunicationCode            timestamp  
0                            60  2022-02-02 00:05:00  
1                            60  2022-02-02 00:10:00  
2                            60  2022-02-02 00:15:00  
3                            60  2022-02-02 00:20:00  
4   

In [ ]:
flag_df = pd.DataFrame(columns=['INV01_CommunicationCode','INV03_CommunicationCode','INV03_CommunicationCode'])
for i in code_df.index:
    if not any(key == 60 for key in code_df.loc[i, 'INV01_CommunicationCode':'INV02_CommunicationCode']):
        code_df.drop(i, axis=0, inplace=True)

In [43]:
code_df.to_csv("./data/comm_code_filtered.csv", sep = ";", index = False)

        Unnamed: 0  INV01_CommunicationCode  INV03_CommunicationCode  \
0                0                       60                       60   
1                1                       60                       60   
2                2                       60                       60   
3                3                       60                       60   
4                4                       60                       60   
...            ...                      ...                      ...   
143425      143425                       60                        0   
143426      143426                       60                        0   
143427      143427                       60                        0   
143428      143428                       60                        0   
143429      143429                       60                        0   

        INV02_CommunicationCode            timestamp  
0                            60  2022-02-02 00:05:00  
1                        

# Old

In [ ]:

for i, col in enumerate(df.columns):
    sorted_arr_test = get_corr(df_res_test)
    print(sorted_arr_test)
    #wandb.init(project="AnomalyDetection")
    #table = wandb.Table(columns = ["mtad_gat"])
    if i <= 20:
    
        trace = []


        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test[f"Forecast_{i}"], name=f"{col} forecast"))
        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test[f"Recon_{i}"], name=f"{col} recon"))
        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test[f"True_{i}"], name=f"{col} true"))
        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test["A_Pred_Global"], name="Global anomalies"))
        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test[f"A_Pred_{i}"], name=f"{col} anomalies"))
        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test[f"Thresh_{i}"], name=f"{col} threshold", line=dict(color='royalblue', dash='dash')))
        trace.append(go.Scatter(x=timestamp_t.values, y=df_res_test[f"A_Score_{i}"], name=f"A_Score_{i}" ,line=dict(color='firebrick',width=1, dash='dot')))

        layout = go.Layout(title= f"{col} Plot", xaxis=dict(title='timestamp'), yaxis=dict(title='value'))
        

        fig = go.Figure(data=trace, layout=layout)

        fig.write_html(f"./graphs/15_sunrise_test/{i}_plot_mtad_gat.html", auto_play = False) 
        # Add Plotly figure as HTML file into Table
        #table.add_data(wandb.Html(f"./graphs/{col}_plot_mtad_gat.html"))
        #wandb.log({f"{col}_plot_mtad_gat": table})
        #wandb.finish()
        
        
        '''
        anomaly_shape = self.create_shapes(
                anomaly_sequences, None, y_min, y_max, None, xref=xref, yref=yref, is_test=is_test
            )
            shapes.extend(anomaly_shape)
        '''
        
        
        
        #fig.show()
    else:
        break

# Correlazione anomalie


In [ ]:
tmp = get_corr(df_res)

In [ ]:
tmp

In [ ]:
sorted_arr